# LG-CoTrain: All Disasters Re-Run

This notebook re-runs the **LG-CoTrain** co-training pipeline across **all 10 disaster
events** with a **configurable pseudo-label source** and stores results in a
**named sub-folder** under `results/`.

### Why a separate notebook?

We may run experiments multiple times with different pseudo-label sets (e.g.,
gpt-4o vs llama-3, or multiple runs of the same model). Each run is stored in
its own sub-folder so results are never overwritten.

### Configuration (Cell 2)

Edit the following variables in the **Configuration** cell before running:

| Variable | Description |
|---|---|
| `PSEUDO_LABEL_SOURCE` | Name of the pseudo-label directory under `data/pseudo-labelled/` |
| `RUN_NAME` | Sub-folder name under `results/` for this run |

### Resume Support

Same two-level resume as notebook 02:

1. **Event level**: Events with all 12 `metrics.json` files are skipped entirely.
2. **Experiment level**: Individual `(budget, seed_set)` combinations with existing
   results are skipped within each event.

In [1]:
import json
import statistics
import sys
import time
from pathlib import Path

def _find_repo_root(marker: str = "lg_cotrain") -> Path:
    for candidate in [Path().resolve()] + list(Path().resolve().parents):
        if (candidate / marker).is_dir():
            return candidate
    raise RuntimeError(
        f"Cannot find repo root: no ancestor directory contains '{marker}/'. "
        "Run the notebook from inside the repository."
    )

repo_root = _find_repo_root()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import matplotlib.pyplot as plt
import numpy as np

from lg_cotrain.run_all import BUDGETS, SEED_SETS, run_all_experiments, format_summary_table

print(f"Repo root: {repo_root}")
print(f"Budgets: {BUDGETS}")
print(f"Seed sets: {SEED_SETS}")
print(f"Experiments per event: {len(BUDGETS) * len(SEED_SETS)}")

Repo root: D:\Workspace\Co-Training
Budgets: [5, 10, 25, 50]
Seed sets: [1, 2, 3]
Experiments per event: 12


In [2]:
# ---- User-editable configuration ----
PSEUDO_LABEL_SOURCE = "gpt-4o"      # Change to use different pseudo-labels (e.g. "llama-3")
RUN_NAME = "gpt-4o-run-2"            # Sub-folder name under results/

DATA_ROOT = str(repo_root / "data")
RESULTS_ROOT = str(repo_root / "results" / RUN_NAME)

print(f"Pseudo-label source: {PSEUDO_LABEL_SOURCE}")
print(f"Run name: {RUN_NAME}")
print(f"Data root: {DATA_ROOT}")
print(f"Results root: {RESULTS_ROOT}")

Pseudo-label source: gpt-4o
Run name: gpt-4o-run-2
Data root: D:\Workspace\Co-Training\data
Results root: D:\Workspace\Co-Training\results\gpt-4o-run-2


In [3]:
def is_event_complete(event, results_root):
    """Check if all 12 metrics.json files exist for an event."""
    for budget in BUDGETS:
        for seed_set in SEED_SETS:
            path = Path(results_root) / event / f"{budget}_set{seed_set}" / "metrics.json"
            if not path.exists():
                return False
    return True

# Discover events from data directory
data_dir = Path(DATA_ROOT) / "original"
all_events = sorted(p.name for p in data_dir.iterdir() if p.is_dir())

completed_events = [e for e in all_events if is_event_complete(e, RESULTS_ROOT)]
pending_events = [e for e in all_events if e not in completed_events]

print(f"Found {len(all_events)} events total")
print(f"  Completed: {len(completed_events)} ({len(completed_events) * 12} experiments)")
print(f"  Pending:   {len(pending_events)} (up to {len(pending_events) * 12} experiments)")

if completed_events:
    print(f"\nCompleted events (will be skipped):")
    for e in completed_events:
        print(f"  - {e}")

if pending_events:
    print(f"\nPending events (will be run):")
    for e in pending_events:
        print(f"  - {e}")

Found 10 events total
  Completed: 0 (0 experiments)
  Pending:   10 (up to 120 experiments)

Pending events (will be run):
  - california_wildfires_2018
  - canada_wildfires_2016
  - cyclone_idai_2019
  - hurricane_dorian_2019
  - hurricane_florence_2018
  - hurricane_harvey_2017
  - hurricane_irma_2017
  - hurricane_maria_2017
  - kaikoura_earthquake_2016
  - kerala_floods_2018


## Running Experiments

For each pending event, we call `run_all_experiments` with the configured
`pseudo_label_source` and `results_root` pointing to our named sub-folder.

Individual experiments that already have `metrics.json` are automatically
skipped (useful if the notebook crashed mid-event).

In [ ]:
class ProgressTracker:
    """Track global progress across all experiments."""

    def __init__(self, total, already_done, start_time):
        self.total = total
        self.done = already_done
        self.start_time = start_time

    def update(self, event, budget, seed_set, status):
        self.done += 1
        elapsed = time.time() - self.start_time
        pct = 100.0 * self.done / self.total
        elapsed_h = elapsed / 3600

        remaining = self.total - self.done
        if elapsed > 0 and self.done > 0:
            eta_h = (elapsed / self.done) * remaining / 3600
        else:
            eta_h = 0

        print(
            f"[PROGRESS] {self.done}/{self.total} ({pct:.1f}%)"
            f" | Elapsed: {elapsed_h:.2f}h | ETA: {eta_h:.2f}h"
        )

# Count already-completed experiments (from previous runs)
already_done = sum(
    1
    for e in all_events
    for b in BUDGETS
    for s in SEED_SETS
    if (Path(RESULTS_ROOT) / e / f"{b}_set{s}" / "metrics.json").exists()
)
total_experiments = len(all_events) * len(BUDGETS) * len(SEED_SETS)

print(f"Total experiments: {total_experiments}")
print(f"Already completed: {already_done}")
print(f"Remaining: {total_experiments - already_done}")

all_event_results = {}
overall_start = time.time()
tracker = ProgressTracker(total_experiments, already_done, overall_start)

# Run pending events
for i, event in enumerate(pending_events, 1):
    print(f"\n{'=' * 60}")
    print(f"Event {i}/{len(pending_events)}: {event}")
    print(f"{'=' * 60}")

    results = run_all_experiments(
        event,
        pseudo_label_source=PSEUDO_LABEL_SOURCE,
        data_root=DATA_ROOT,
        results_root=RESULTS_ROOT,
        _on_experiment_done=tracker.update,
    )
    all_event_results[event] = results

    print()
    print(format_summary_table(results, event))

# Load results for already-completed events
for event in completed_events:
    results = []
    for budget in BUDGETS:
        for seed_set in SEED_SETS:
            path = Path(RESULTS_ROOT) / event / f"{budget}_set{seed_set}" / "metrics.json"
            with open(path) as f:
                results.append(json.load(f))
    all_event_results[event] = results

overall_elapsed = time.time() - overall_start
print(f"\n{'=' * 60}")
print(f"All events done in {overall_elapsed / 3600:.2f}h")
print(f"Total events with results: {len(all_event_results)}")

Total experiments: 120
Already completed: 0
Remaining: 120

Event 1/10: california_wildfires_2018


D:\Workspace\Co-Training\co-training-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[1/12] budget=5, seed=1 -- starting...


2026-02-18 13:26:52,260 - lg_cotrain - INFO - Starting LG-CoTrain: event=california_wildfires_2018, budget=5, seed_set=1
2026-02-18 13:26:52,292 - lg_cotrain - INFO - Detected 10 classes for event california_wildfires_2018: ['caution_and_advice', 'displaced_people_and_evacuations', 'infrastructure_and_utility_damage', 'injured_or_dead_people', 'missing_or_found_people', 'not_humanitarian', 'other_relevant_information', 'requests_or_urgent_needs', 'rescue_volunteering_or_donation_effort', 'sympathy_and_support']
2026-02-18 13:26:52,303 - lg_cotrain - INFO - D_l1: 30, D_l2: 20, D_LG: 5113
2026-02-18 13:26:52,305 - lg_cotrain - INFO - === Phase 1: Weight Generation ===
Loading weights: 100%|███████████████| 199/199 [00:00<00:00, 1091.60it/s, Materializing param=bert.pooler.dense.weight]
2026-02-18 13:27:10,883 - lg_cotrain - INFO - Phase 1 epoch 1/7: mean_prob1=0.0981, mean_prob2=0.1025
2026-02-18 13:27:28,265 - lg_cotrain - INFO - Phase 1 epoch 2/7: mean_prob1=0.0997, mean_prob2=0.0991
2

[1/12] budget=5, seed=1 -- done (macro_f1=0.6405)
[PROGRESS] 1/120 (0.8%) | Elapsed: 0.25h | ETA: 29.76h
[2/12] budget=5, seed=2 -- starting...


...

2026-02-18 18:07:34,560 - lg_cotrain - INFO - Starting LG-CoTrain: event=cyclone_idai_2019, budget=25, seed_set=2
2026-02-18 18:07:34,601 - lg_cotrain - INFO - Detected 10 classes for event cyclone_idai_2019: ['caution_and_advice', 'displaced_people_and_evacuations', 'infrastructure_and_utility_damage', 'injured_or_dead_people', 'missing_or_found_people', 'not_humanitarian', 'other_relevant_information', 'requests_or_urgent_needs', 'rescue_volunteering_or_donation_effort', 'sympathy_and_support']
2026-02-18 18:07:34,609 - lg_cotrain - INFO - D_l1: 124, D_l2: 114, D_LG: 2515
2026-02-18 18:07:34,610 - lg_cotrain - INFO - === Phase 1: Weight Generation ===
Loading weights: 100%|███████████████| 199/199 [00:00<00:00, 1240.80it/s, Materializing param=bert.pooler.dense.weight]
2026-02-18 18:07:45,462 - lg_cotrain - INFO - Phase 1 epoch 1/7: mean_prob1=0.1017, mean_prob2=0.1019
2026-02-18 18:07:55,236 - lg_cotrain - INFO - Phase 1 epoch 2/7: mean_prob1=0.1041, mean_prob2=0.1150
2026-02-18 18:

[8/12] budget=25, seed=2 -- done (macro_f1=0.6023)
[PROGRESS] 32/120 (26.7%) | Elapsed: 4.80h | ETA: 13.21h
[9/12] budget=25, seed=3 -- starting...


2026-02-18 18:15:00,748 - lg_cotrain - INFO - Starting LG-CoTrain: event=cyclone_idai_2019, budget=25, seed_set=3
2026-02-18 18:15:00,786 - lg_cotrain - INFO - Detected 10 classes for event cyclone_idai_2019: ['caution_and_advice', 'displaced_people_and_evacuations', 'infrastructure_and_utility_damage', 'injured_or_dead_people', 'missing_or_found_people', 'not_humanitarian', 'other_relevant_information', 'requests_or_urgent_needs', 'rescue_volunteering_or_donation_effort', 'sympathy_and_support']
2026-02-18 18:15:00,792 - lg_cotrain - INFO - D_l1: 124, D_l2: 114, D_LG: 2515
2026-02-18 18:15:00,793 - lg_cotrain - INFO - === Phase 1: Weight Generation ===
Loading weights: 100%|███████████████| 199/199 [00:00<00:00, 1190.31it/s, Materializing param=bert.pooler.dense.weight]
2026-02-18 18:15:11,680 - lg_cotrain - INFO - Phase 1 epoch 1/7: mean_prob1=0.1000, mean_prob2=0.1077
2026-02-18 18:15:21,442 - lg_cotrain - INFO - Phase 1 epoch 2/7: mean_prob1=0.1040, mean_prob2=0.1088
2026-02-18 18:

[9/12] budget=25, seed=3 -- done (macro_f1=0.6124)
[PROGRESS] 33/120 (27.5%) | Elapsed: 4.93h | ETA: 12.99h
[10/12] budget=50, seed=1 -- starting...


2026-02-18 18:22:24,949 - lg_cotrain - INFO - Starting LG-CoTrain: event=cyclone_idai_2019, budget=50, seed_set=1
2026-02-18 18:22:24,971 - lg_cotrain - INFO - Detected 10 classes for event cyclone_idai_2019: ['caution_and_advice', 'displaced_people_and_evacuations', 'infrastructure_and_utility_damage', 'injured_or_dead_people', 'missing_or_found_people', 'not_humanitarian', 'other_relevant_information', 'requests_or_urgent_needs', 'rescue_volunteering_or_donation_effort', 'sympathy_and_support']
2026-02-18 18:22:24,971 - lg_cotrain - INFO - D_l1: 227, D_l2: 226, D_LG: 2300
2026-02-18 18:22:24,971 - lg_cotrain - INFO - === Phase 1: Weight Generation ===
Loading weights: 100%|███████████████| 199/199 [00:00<00:00, 1216.71it/s, Materializing param=bert.pooler.dense.weight]
2026-02-18 18:22:36,569 - lg_cotrain - INFO - Phase 1 epoch 1/7: mean_prob1=0.1059, mean_prob2=0.1254
2026-02-18 18:22:46,861 - lg_cotrain - INFO - Phase 1 epoch 2/7: mean_prob1=0.1102, mean_prob2=0.1264
2026-02-18 18:

## Cross-Disaster Results

We now aggregate results across all events to compare how the pipeline
performs on different disaster types and how performance scales with the
labeled data budget.

In [ ]:
# Build cross-disaster summary: event -> budget -> mean macro-F1
summary = {}
for event in sorted(all_event_results.keys()):
    results = all_event_results[event]
    by_budget = {b: [] for b in BUDGETS}
    for r in results:
        if r is not None:
            by_budget[r["budget"]].append(r)
    summary[event] = {}
    for b in BUDGETS:
        f1s = [r["test_macro_f1"] for r in by_budget[b]]
        errs = [r["test_error_rate"] for r in by_budget[b]]
        summary[event][b] = {
            "f1_mean": statistics.mean(f1s) if f1s else None,
            "f1_std": statistics.stdev(f1s) if len(f1s) >= 2 else None,
            "err_mean": statistics.mean(errs) if errs else None,
            "err_std": statistics.stdev(errs) if len(errs) >= 2 else None,
            "n_seeds": len(f1s),
        }

# Print grand summary table
header = f"{'Event':<35}"
for b in BUDGETS:
    header += f" | B={b:<11}"
print(header)
print("-" * len(header))

for event in sorted(summary.keys()):
    row = f"{event:<35}"
    for b in BUDGETS:
        s = summary[event][b]
        if s["f1_mean"] is not None and s["f1_std"] is not None:
            row += f" | {s['f1_mean']:.3f}+/-{s['f1_std']:.3f}"
        elif s["f1_mean"] is not None:
            row += f" | {s['f1_mean']:.3f}      "
        else:
            row += f" | {'N/A':<11}"
    print(row)

# Line plot: Macro-F1 by budget, one line per event
fig, ax = plt.subplots(figsize=(10, 6))

for event in sorted(summary.keys()):
    means = [summary[event][b]["f1_mean"] or 0 for b in BUDGETS]
    stds = [summary[event][b]["f1_std"] or 0 for b in BUDGETS]
    ax.errorbar(BUDGETS, means, yerr=stds, marker="o", capsize=3, label=event)

ax.set_xlabel("Budget (labeled samples per class)")
ax.set_ylabel("Test Macro-F1 (mean +/- std across seeds)")
ax.set_title(f"LG-CoTrain Performance — {PSEUDO_LABEL_SOURCE}")
ax.set_xticks(BUDGETS)
ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Heatmap: events (rows) x budgets (columns), colored by mean macro-F1
events_sorted = sorted(summary.keys())
heatmap_data = np.zeros((len(events_sorted), len(BUDGETS)))

for i, event in enumerate(events_sorted):
    for j, b in enumerate(BUDGETS):
        val = summary[event][b]["f1_mean"]
        heatmap_data[i, j] = val if val is not None else 0

fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(heatmap_data, cmap="YlOrRd", aspect="auto")

ax.set_xticks(range(len(BUDGETS)))
ax.set_xticklabels([f"B={b}" for b in BUDGETS])
ax.set_yticks(range(len(events_sorted)))
ax.set_yticklabels(events_sorted, fontsize=9)
ax.set_title(f"Mean Test Macro-F1 — {PSEUDO_LABEL_SOURCE}")

for i in range(len(events_sorted)):
    for j in range(len(BUDGETS)):
        val = heatmap_data[i, j]
        color = "white" if val > 0.6 else "black"
        ax.text(j, i, f"{val:.3f}", ha="center", va="center", color=color, fontsize=9)

fig.colorbar(im, ax=ax, label="Macro-F1")
plt.tight_layout()
plt.show()

In [ ]:
from lg_cotrain.dashboard import collect_all_metrics, generate_html

metrics = collect_all_metrics(RESULTS_ROOT)
html = generate_html(metrics, RESULTS_ROOT)
dashboard_path = Path(RESULTS_ROOT) / "dashboard.html"
dashboard_path.parent.mkdir(parents=True, exist_ok=True)
dashboard_path.write_text(html)
print(f"Dashboard written to: {dashboard_path}")
print(f"Metrics loaded: {len(metrics)} experiments")

## Summary

This notebook ran experiments for all disaster events using:
- **Pseudo-label source**: configured via `PSEUDO_LABEL_SOURCE`
- **Results folder**: `results/{RUN_NAME}/`

Results are stored separately from previous runs, enabling side-by-side
comparison via the multi-tab dashboard.

### CLI equivalent
```bash
# Run all experiments for all events with custom pseudo-label source and output folder
python -m lg_cotrain.run_experiment \
    --events california_wildfires_2018 canada_wildfires_2016 cyclone_idai_2019 \
            hurricane_dorian_2019 hurricane_florence_2018 hurricane_harvey_2017 \
            hurricane_irma_2017 hurricane_maria_2017 kaikoura_earthquake_2016 \
            kerala_floods_2018 \
    --pseudo-label-source gpt-4o \
    --output-folder results/gpt-4o-run1
```

### Generate multi-tab dashboard from the CLI
```bash
python -m lg_cotrain.dashboard --results-root results/
```